In [1]:
from collections import defaultdict
import time
import random
import numpy as np
import os
import torch
from torch import nn
from torch.autograd import Variable
from models import BoW

In [2]:
# read corpus
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))

UNK = w2i["<unk>"]
def read_dataset(filename):
    with open(filename, "r") as f:
        for line in f:
            tag, words = line.lower().strip().split(" ||| ")
            yield ([w2i[x] for x in words.split(" ")], t2i[tag])

In [3]:
# read data
train = list(read_dataset("../data/classes/train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("../data/classes/test.txt"))
nwords = len(w2i)
ntags = len(t2i)

In [4]:
# initialise the model
model = BoW(nwords, ntags)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [5]:
for ITER in range(100):
    random.shuffle(train)
    train_loss = 0.0
    start = time.time()
    for words, tag in train:
        words = torch.Tensor(words).type(torch.LongTensor)
        tag = torch.Tensor([tag]).type(torch.LongTensor)
        scores = model(words)
        loss = loss_fn(scores, tag)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("iter %r: train loss/sent=%.4f, time=%.2fs" % (
                ITER, train_loss/len(train), time.time()-start))
    test_correct = 0
    for words, tag in dev:
        words = torch.Tensor(words).type(torch.LongTensor)
        tag = torch.Tensor([tag]).type(torch.LongTensor)
        scores = model(words)[0].detach().cpu().numpy()
        predict = np.argmax(scores)
        if predict == tag:
            test_correct += 1
    print("iter %r: test acc=%.4f" % (ITER, test_correct/len(dev)))

iter 0: train loss/sent=1.4721, time=14.93s
iter 0: test acc=0.3914
iter 1: train loss/sent=1.1213, time=17.79s
iter 1: test acc=0.4154
iter 2: train loss/sent=0.9118, time=17.73s
iter 2: test acc=0.4145
iter 3: train loss/sent=0.7692, time=17.74s
iter 3: test acc=0.4118
iter 4: train loss/sent=0.6630, time=17.78s
iter 4: test acc=0.4054
iter 5: train loss/sent=0.5818, time=17.84s
iter 5: test acc=0.4104
iter 6: train loss/sent=0.5164, time=17.95s
iter 6: test acc=0.4081
iter 7: train loss/sent=0.4634, time=18.34s
iter 7: test acc=0.4005
iter 8: train loss/sent=0.4192, time=17.87s
iter 8: test acc=0.4014
iter 9: train loss/sent=0.3809, time=18.17s
iter 9: test acc=0.3932
iter 10: train loss/sent=0.3489, time=18.55s
iter 10: test acc=0.3941
iter 11: train loss/sent=0.3209, time=18.21s
iter 11: test acc=0.3919
iter 12: train loss/sent=0.2958, time=17.90s
iter 12: test acc=0.3887
iter 13: train loss/sent=0.2741, time=18.03s
iter 13: test acc=0.3882
iter 14: train loss/sent=0.2552, time=17